# Computer Science Problems in Python

## fibonacci sample
### using dict for memoization

To avoid stack overflow errors and increase performance during recursive method call we use a technique, called memoization.

In [1]:
from typing import Dict

memo: Dict[int, int] = {0:0, 1:1}

def fib1(n: int) -> int:
    if n not in memo:
        memo[n] = fib1(n-1) + fib1(n-2)
    return memo[n]
    
print(fib1(5))
print(fib1(50))

5
12586269025


### using decorator for memoization

In [2]:
from functools import lru_cache

@lru_cache(maxsize=None)
def fib2(n: int) -> int:
    if n < 2:
        return n
    return fib2(n-1) + fib2(n-2)

print(fib2(5))
print(fib2(50))

5
12586269025


### iterative style

In [3]:
from typing import Generator

def fib3(n: int) -> Generator[int, None, None]:
    if n == 0: return n
    
    last: int = 0
    next: int = 1

    for _ in range(1, n):
        last, next = next, last + next #variable swap
        
    return next

print(fib3(5))
print(fib3(50))

5
12586269025


### print each number

In [4]:
def fib4(n: int) -> Generator[int, None, None]:
    yield 0
    if n > 0: yield 1
    
    last: int = 0
    next: int = 1

    for _ in range(1, n):
        last, next = next, last + next
        yield next
        
for i in fib4(5):
        print(i)

0
1
1
2
3
5


## Sample of compression: nucleotide (ACGT) stored as bit in stead of string

* A = 00
* C = 01
* G = 10
* T = 11

In [5]:
class CompressedGene:
    def __init__(self, gene: str) -> None:
        self._compress(gene)

    def _compress(self, gene: str) -> None:
        self.bit_string: int = 1  # start with sentinel
        for nucleotide in gene.upper():
            self.bit_string <<= 2  # shift left two bits
            if nucleotide == "A":  # change last two bits to 00
                self.bit_string |= 0b00
            elif nucleotide == "C":  # change last two bits to 01
                self.bit_string |= 0b01
            elif nucleotide == "G":  # change last two bits to 10
                self.bit_string |= 0b10
            elif nucleotide == "T":  # change last two bits to 11
                self.bit_string |= 0b11
            else:
                raise ValueError("Invalid nucleotide:{}".format(nucleotide))

    def decompress(self) -> str:
        gene: str = ""
        for i in range(0, self.bit_string.bit_length() - 1, 2):  # - 1 to exclude sentinel
            bits: int = self.bit_string >> i & 0b11  # get just 2 relevant bits
            if bits == 0b00:  # A
                gene += "A"
            elif bits == 0b01:  # C
                gene += "C"
            elif bits == 0b10:  # G
                gene += "G"
            elif bits == 0b11:  # T
                gene += "T"
            else:
                raise ValueError("Invalid bits:{}".format(bits))
        return gene[::-1]  # [::-1] reverses string by slicing backward

    def __str__(self) -> str:  # string representation for pretty printing
        return self.decompress()
    

from sys import getsizeof

original: str = "TAGGGATTAACCGTTATATATATATAGCCATGGATCGATTATA" * 10
print("original is {} bytes".format(getsizeof(original)))

compressed: CompressedGene = CompressedGene(original)  # compress
print("compressed is {} bytes".format(getsizeof(compressed.bit_string)))

# decompress
print(compressed)  

print("original and decompressed are the same: {}".format(original == compressed.decompress()))

original is 479 bytes
compressed is 140 bytes
TAGGGATTAACCGTTATATATATATAGCCATGGATCGATTATATAGGGATTAACCGTTATATATATATAGCCATGGATCGATTATATAGGGATTAACCGTTATATATATATAGCCATGGATCGATTATATAGGGATTAACCGTTATATATATATAGCCATGGATCGATTATATAGGGATTAACCGTTATATATATATAGCCATGGATCGATTATATAGGGATTAACCGTTATATATATATAGCCATGGATCGATTATATAGGGATTAACCGTTATATATATATAGCCATGGATCGATTATATAGGGATTAACCGTTATATATATATAGCCATGGATCGATTATATAGGGATTAACCGTTATATATATATAGCCATGGATCGATTATATAGGGATTAACCGTTATATATATATAGCCATGGATCGATTATA
original and decompressed are the same: True


## Encryption

A one-time pad is a way of encrypting a piece of data by combining it with meaningless 
random dummy data in such a way that the original cannot be reconstituted without access 
to both the product and the dummy data. In essence, this leaves the encrypter with a key 
pair. One key is the product, and the other is the random dummy data.

In [6]:
from secrets import token_bytes
from typing import Tuple

def random_key(length: int) -> int:
    # generate length random bytes
    tb: bytes = token_bytes(length)
    # convert those bytes into a bit string and return it
    return int.from_bytes(tb, "big")

def encrypt(original: str) -> Tuple[int, int]:
    original_bytes: bytes = original.encode()
    dummy: int = random_key(len(original_bytes))
    original_key: int = int.from_bytes(original_bytes, "big")
    encrypted: int = original_key ^ dummy  # XOR
    return dummy, encrypted

def decrypt(key1: int, key2: int) -> str:
    decrypted: int = key1 ^ key2  # XOR
    temp: bytes = decrypted.to_bytes((decrypted.bit_length()+ 7) // 8, "big")
    return temp.decode()


key1, key2 = encrypt("One Time Pad!") #-> dummy data, product
print(key1, key2)
result: str = decrypt(key1, key2)
print(result)

1329362375362379684150920081274 7578983132105860114807361276763
One Time Pad!


## The Number PI

Leibniz formula to derive PI:
π = 4/1 - 4/3 + 4/5 - 4/7 + 4/9 - 4/11...

In [7]:
def pi(n_terms: int) -> float:
    numerator: float = 4.0 #constant

    denominator: float = 1.0
    operation: float = 1.0
    pi: float = 0.0
    for _ in range(n_terms):
        pi += operation * (numerator / denominator)
        denominator += 2.0
        operation *= -1.0
    return pi

print(pi(1000000))

3.1415916535897743
